In [1]:
import numpy as np
import pandas as pd
from main import run_di, run_ehd, experiments
from sim_env import FixedSimEnv
from objects import td

In [ ]:
# pax, trips, events, wlogs = experiments()

In [2]:
env = FixedSimEnv()
unique_dates = env.link_times['date'].unique()
dates = np.random.choice(unique_dates, replace=False, size=3)

In [3]:
next_obs, rew, done, info = env.reset(hist_date='2022-11-21')
print(env.hist_date)

2022-11-21


In [7]:
env, info, n_steps, ref_veh_df, rts_info, borrow_rts = run_di(env, done, debug=True)
print(td(env.time), n_steps)

while loop:  11%|█▏        | 1355/12000 [00:24<03:15, 54.47it/s]

0 days 06:36:57 1355


In [8]:
rts_info['91']

,route_id,trip_sequence,runid,block_id,trip_id,shape_id,confirmed,ST_str,ST,AT,DT
0,91,7,F111,652000002595,28414020,65202945,1,06:22:30,22950.0,23041.230265,23054.992101
1,91,8,F161,652000002545,16130020,65202945,1,06:32:00,23520.0,23760.000000,23775.794019
2,91,9,F166,652000002631,39674020,65202945,0,06:41:30,24090.0,NaN,NaN
3,91,10,F167,652000002581,38441020,65202945,0,07:01:00,25260.0,NaN,NaN
4,91,11,F162,652000002632,48610020,65202945,1,07:11:00,25860.0,NaN,NaN
5,91,13,F169,652000002541,37199020,65202945,1,07:32:30,27150.0,NaN,NaN
6,91,14,F164,652000002603,4650020,65202945,0,07:43:30,27810.0,NaN,NaN
7,91,15,F168,652000002561,12758020,65202945,1,07:54:30,28470.0,NaN,NaN
8,91,16,F111,652000002595,37542020,65202945,1,08:06:00,29160.0,NaN,NaN


In [13]:
from interlining import *

In [14]:
def pred_inbound_tt(veh, line, start_time):
    # compute time until arrival at terminal
    trip = veh.curr_trip
    stop_idx_from = veh.stop_idx ## this is zero-starting
    stop_idx_to = len(trip.stops) - 1
    travel_time = line.time_between_two_stops(stop_idx_from, stop_idx_to, 
                                                trip.stops, start_time)
    return travel_time


def pred_terminal_report_time(next_trips, idx, info_vehs, env):
    block_id = next_trips.loc[idx, 'block_id']

    ## this is based on the info from env.step output
    veh_info = info_vehs[
        info_vehs['block_id']==block_id].iloc[[0]].copy()
    
    ## this is the object
    veh = env.vehicles[veh_info.index[0]]

    ## other variables
    time = env.time
    line = env.lines[veh.route_id]
    schd_dep = next_trips.loc[idx, 'ST']

    next_event_t = veh_info['next_event_t'].iloc[0].total_seconds()
    status = veh_info['status'].iloc[0]
    direct = veh_info['direction'].iloc[0]
    rt = veh_info['route_id'].iloc[0]
    earliest_dep = schd_dep-MAX_EARLY_DEV*60
    
    assert status in [0,1,2,3,4,5]

    if status in [0,5]: # yet to report
        if direct == OUTBOUND_DIRECTIONS[rt]:
            # the report time at initialization
            # ensure next event is within OTP
            return max(next_event_t, earliest_dep)
        else:
            return schd_dep
    
    if status in [1,4]: 
        if direct == OUTBOUND_DIRECTIONS[rt]:
            # reported (and adjusted)
            return next_event_t
        else:
            return schd_dep # scheduled departure is as good a guess as it gets
    
    # dwell time status (2) at terminal is not considered
    # trips dwelling at terminal would have been flagged as
    # last trip in stop.report_last

    if status in [2,3]:
        if direct == OUTBOUND_DIRECTIONS[rt]:
            return schd_dep # scheduled departure is as good a guess as it gets
        else:
            start_time = time
            travel_time = pred_inbound_tt(veh, line, start_time)

            arrives = time + travel_time
            return max(arrives, earliest_dep)

In [15]:
def eval_interlining(info, control_vehs, rts_info, borrow_rts, env,
                     debug=False):
    lend_rt_id = control_vehs['route_id'].iloc[0]

    scenarios_L, vars_L, next_info_L = lend_variables(rts_info[lend_rt_id], 
                                                    info[1], env, control_vehs)

    impact = {lend_rt_id: change_in_hw(vars_L)}

    ## initial conditions
    best_switch = {'ratio': 1.2, 
                   'lend_trip_ids': next_info_L['trip_ids'],
                   'route': None,
                   'departure': None,
                   'schedules': None,
                   'scenarios': {'lend': scenarios_L, 'borrow': None},
                   'variables': {'lend': vars_L, 'borrow': None},
                   'est_return': None,
                   'req_return': next_info_L['departure']}

    for rt in borrow_rts:
        rt_info = rts_info[rt].copy()
        scenarios_B, vars_B, next_info_B = borrow_variables(rt_info, info[1], env)
        req_return = next_info_L['departure']
        est_return = next_info_B['return']

        if debug:
            print(lend_rt_id, rt)
            print(req_return, est_return)

        if req_return is None or req_return >= est_return:
            benefit = change_in_hw(vars_B)
            cost = impact[lend_rt_id]
            ratio = benefit/cost
            
            if debug:
                print(lend_rt_id, rt)
                print(req_return, est_return)
                print(benefit, cost)
                print(ratio)
            ## is ghost return also missing?
            schds = next_info_B['schedules']
            both_missing = schds['confirmed'].unique().shape[0] == 1

            if ratio > best_switch['ratio'] and both_missing:
                best_switch['route'] = deepcopy(rt)
                best_switch['ratio'] = deepcopy(ratio)
                best_switch['schedules'] = next_info_B['schedules'].copy()
                next_dep = vars_B.loc[
                    vars_B['var']=='next departure', 'counter'].iloc[0]
                best_switch['departure'] = next_dep
                best_switch['scenarios']['borrow'] = scenarios_B
                best_switch['variables']['borrow'] = vars_B
                best_switch['est_return'] = est_return
                best_switch['req_return'] = req_return
    return best_switch

def predict_with_trip(base_df, env, info_vehs, sub_cols, rt_id,
                      borrower=False):
    ## base df is the set of next trips
    scenario = base_df.copy()
    if borrower:
        scenario.loc[2, 'confirmed'] = 1
        scenario = scenario[scenario['confirmed']==1]
        scenario = scenario.reset_index(drop=True)

    ## report time is now if lender
    if borrower:
        max_time = scenario.loc[2, 'ST']-MAX_EARLY_DEV*60
        scenario.loc[2, 'RT'] = max(env.time, max_time)
    else:
        scenario.loc[2,'RT'] = env.time
        
    trip_id = scenario['trip_id'].iloc[2]
    _, max_dep_t = env.terminal_dep_limits(rt_id, trip_id)
    scenario.loc[2, 'DT_max'] = max_dep_t ## ensure limits

    ## we just need to predict the next trip
    report_time = pred_terminal_report_time(
        scenario, 3, info_vehs, env) 
    scenario.loc[3, 'RT'] = report_time

    return scenario[sub_cols]

def predict_without_trip(base_df, env, info_vehs, sub_cols, rt_id,
                         lender=False):
    scenario = base_df.copy()

    ## this is exclusive to the lender case
    if lender:
        scenario.loc[2, 'confirmed'] = 0

    ## exclude all missing trips
    scenario = scenario[scenario['confirmed']==1].copy()
    scenario = scenario.reset_index(drop=True)

    ## just record departure limit, report time later
    trip_id = scenario['trip_id'].iloc[2]
    _, max_dep_t = env.terminal_dep_limits(rt_id, trip_id)
    scenario.loc[2, 'DT_max'] = max_dep_t
    
    for j in range(2, 4):
        report_time = pred_terminal_report_time(
            scenario, j, info_vehs, env) 
        scenario.loc[j, 'RT'] = report_time        

    return scenario[sub_cols]

def lend_variables(rt_info, veh_info, env, control_vehs):
    ## first, create baseline
    base = rt_info[rt_info['confirmed']==1].copy()
    base = base.reset_index(drop=True)
    base[['RT', 'DT_max']] = np.nan

    ## collect useful information
    rt_id = base['route_id'].iloc[0]
    schd = env.routes[rt_id].schedule.copy()
    trip_seq = base['trip_sequence'].iloc[2]
    schd_hw = get_schd_hw(schd, rt_id, trip_seq)

    sub_cols = ['route_id','trip_sequence', 
                'ST_str', 'confirmed', 'ST', 
                'AT', 'DT', 'RT', 'DT_max', 'trip_id', 'block_id']
   
    ## get next trips for actual scenario (with trip)
    actual = predict_with_trip(base, env, veh_info, 
                               sub_cols, rt_id)
    actual_vars = compute_variables(actual)

    ## get next trips for counterfactual scenario (without trip)
    counter = predict_without_trip(base, env, veh_info,
                                   sub_cols, rt_id, lender=True)
    counter_vars = compute_variables(counter)

    ## write it all up
    dec_vars = write_decision_vars(actual_vars, counter_vars)
    dec_vars['route_id'] = rt_id
    dec_vars['schd_hw'] = schd_hw

    scenarios = {'actual': actual,
                 'counterfactual': counter}

    ## 
    veh = env.vehicles[control_vehs.index[0]]

    ## find required return time to depart on-time
    next_req_dep_t = None
    next_trips = veh.next_trips[:min(2, len(veh.next_trips))]
    next_trip_ids = [t.id for t in next_trips] #TO-DO: revise?
    if len(veh.next_trips) > 2:
        t_schd = veh.next_trips[2].schedule.copy()
        next_req_dep_t = t_schd['departure_sec'].iloc[0]
    
    next_info = {'departure': next_req_dep_t,
                 'trip_ids': next_trip_ids}
    
    return scenarios, dec_vars, next_info

def borrow_variables(rt_info, info_vehs, env): 
    # info_vehs is info[1] from main

    ## first create baseline, remove later cancelled trips
    base = rt_info[~((rt_info.index>2) & 
                     (rt_info['confirmed']==0))].copy()
    base = base.reset_index(drop=True)
    base[['RT', 'DT_max']]= np.nan

    ## collect useful information
    rt_id = base['route_id'].iloc[0]
    schd = env.routes[rt_id].schedule.copy()
    trip_seq = base['trip_sequence'].iloc[2]
    schd_hw = get_schd_hw(schd, rt_id, trip_seq)

    sub_cols = ['route_id','trip_sequence', 
                'ST_str', 'confirmed', 'ST', 
                'AT', 'DT', 'RT', 'DT_max', 
                'trip_id', 'block_id']
    
    ## predict current scenario (without trip)

    actual = predict_without_trip(base, env, info_vehs, 
                                  sub_cols, rt_id)
    actual_vars = compute_variables(actual)

    ## counterfactual (with trip): 
    ## idx 2 cancelled, idx > 2 not
    # only report time of index=3
    counter = predict_with_trip(base, env, info_vehs, sub_cols,
                                rt_id, borrower=True)
    counter_vars = compute_variables(counter)

    ## write it all up
    dec_vars = write_decision_vars(actual_vars, counter_vars)
    dec_vars['route_id'] = rt_id
    dec_vars['schd_hw'] = schd_hw
    
    scenarios = {'actual': actual,
                 'counterfactual': counter}

    # at what time would it come back to terminal?
    ghost_block = counter.loc[2, 'block_id']
    ghost_trip = counter.loc[2, 'trip_id']
    ghost_arr_t, ghost_schds = get_next_schedules(
        rt_id, ghost_block, ghost_trip, env)
    
    next_info = {'return': ghost_arr_t,
                 'schedules': ghost_schds}
    return scenarios, dec_vars, next_info

In [9]:
info[1][info[1]['block_id']==652000002541]

,time,nr_step,route_id,block_id,direction,active,status,status_desc,next_event,next_event_t,t_until_next,stop_id,stop_sequence,pax_load,t_since_last,trip_id,trip_sequence
21,0 days 06:36:56.751052139,2295,91,652000002541,North,1,3,active - between stops,arrive at stop,0 days 06:39:08.790843895,0 days 00:02:12.039791756,9620,4,3,0 days 00:00:06.960208244,17504020,7


In [17]:
best_swtich = eval_interlining(info, ref_veh_df, rts_info, borrow_rts, env)

In [18]:
best_swtich.keys()

dict_keys(['ratio', 'lend_trip_ids', 'route', 'departure', 'schedules', 'scenarios', 'variables', 'est_return', 'req_return'])

In [25]:
best_swtich['scenarios']['lend']['actual']

,route_id,trip_sequence,ST_str,confirmed,ST,AT,DT,RT,DT_max,trip_id,block_id
0,81,8,06:15:00,1,22500.0,22581.682793,22588.055894,NaN,NaN,44801020,652000002775
1,81,9,06:23:30,1,23010.0,23249.027947,23260.400146,NaN,NaN,20296020,652000002772
2,81,11,06:38:30,1,23910.0,NaN,NaN,23816.751052,24150.0,7518020,652000002769
3,81,12,06:45:30,1,24330.0,NaN,NaN,24392.564580,NaN,24499020,652000002765
4,81,14,06:57:30,1,25050.0,NaN,NaN,NaN,NaN,43223020,652000002735
5,81,15,07:03:30,1,25410.0,NaN,NaN,NaN,NaN,43965020,652000002773
6,81,16,07:10:30,1,25830.0,NaN,NaN,NaN,NaN,4552020,652000002731
7,81,17,07:18:00,1,26280.0,NaN,NaN,NaN,NaN,4712020,652000002763


In [27]:
best_swtich['scenarios']['lend']['counterfactual']

,route_id,trip_sequence,ST_str,confirmed,ST,AT,DT,RT,DT_max,trip_id,block_id
0,81,8,06:15:00,1,22500.0,22581.682793,22588.055894,NaN,NaN,44801020,652000002775
1,81,9,06:23:30,1,23010.0,23249.027947,23260.400146,NaN,NaN,20296020,652000002772
2,81,12,06:45:30,1,24330.0,NaN,NaN,24392.564580,24570.0,24499020,652000002765
3,81,14,06:57:30,1,25050.0,NaN,NaN,25069.929917,NaN,43223020,652000002735
4,81,15,07:03:30,1,25410.0,NaN,NaN,NaN,NaN,43965020,652000002773
5,81,16,07:10:30,1,25830.0,NaN,NaN,NaN,NaN,4552020,652000002731
6,81,17,07:18:00,1,26280.0,NaN,NaN,NaN,NaN,4712020,652000002763


In [28]:
best_swtich['scenarios']['borrow']['actual']


,route_id,trip_sequence,ST_str,confirmed,ST,AT,DT,RT,DT_max,trip_id,block_id
0,91,7,06:22:30,1,22950.0,23041.230265,23054.992101,NaN,NaN,28414020,652000002595
1,91,8,06:32:00,1,23520.0,23760.000000,23775.794019,NaN,NaN,16130020,652000002545
2,91,11,07:11:00,1,25860.0,NaN,NaN,25680.0,26100.0,48610020,652000002632
3,91,13,07:32:30,1,27150.0,NaN,NaN,26970.0,NaN,37199020,652000002541
4,91,15,07:54:30,1,28470.0,NaN,NaN,NaN,NaN,12758020,652000002561
5,91,16,08:06:00,1,29160.0,NaN,NaN,NaN,NaN,37542020,652000002595


In [30]:
best_swtich['scenarios']['borrow']['counterfactual']

,route_id,trip_sequence,ST_str,confirmed,ST,AT,DT,RT,DT_max,trip_id,block_id
0,91,7,06:22:30,1,22950.0,23041.230265,23054.992101,NaN,NaN,28414020,652000002595
1,91,8,06:32:00,1,23520.0,23760.000000,23775.794019,NaN,NaN,16130020,652000002545
2,91,9,06:41:30,1,24090.0,NaN,NaN,23910.0,24330.0,39674020,652000002631
3,91,11,07:11:00,1,25860.0,NaN,NaN,25680.0,NaN,48610020,652000002632
4,91,13,07:32:30,1,27150.0,NaN,NaN,NaN,NaN,37199020,652000002541
5,91,15,07:54:30,1,28470.0,NaN,NaN,NaN,NaN,12758020,652000002561
6,91,16,08:06:00,1,29160.0,NaN,NaN,NaN,NaN,37542020,652000002595


In [31]:
best_swtich['departure']

24330.0

In [4]:
env, info, n_steps, upd_info = run_di(env, done, debug_ehd=True)
print(td(env.time), n_steps)

while loop:   0%|          | 16/12000 [00:00<04:18, 46.33it/s]

----
Time 0 days 05:15:18
Route ID 81
Expected: 0 days 05:17:00
Headways: 0 days 00:08:23 -- 0 days 00:13:00
Schedule: 0 days 05:20:00
0 days 05:17:00 < Departure < 0 days 05:24:00
Recommended 0 days 05:19:18 ---> 0 days 05:19:18
0 days 05:15:18 16


In [29]:
env, info, n_steps, ref_veh_df, rts_info, best_switch = run_di(env, done, debug=True)
print(td(env.time), n_steps)

while loop:  13%|█▎        | 1590/12000 [00:29<03:15, 53.25it/s]

92 81
34110.0 33690.0
92 81
34110.0 33690.0
0.7063422983699943 0.6980647275989049
1.0118578842961472
0 days 07:42:30 1590


In [12]:
from interlining import *

In [29]:
env, info, n_steps, upd_info = run_ehd(env, done, n_steps=n_steps, debug=True)
n_steps

----
Time 0 days 05:55:48
Expected: 0 days 05:56:30
Headways: 0 days 00:08:32 -- 0 days 00:09:00
Schedule: 0 days 05:56:30
0 days 05:55:48 < Departure < 0 days 06:01:30
Recommended 0 days 05:56:44 ---> 0 days 05:56:44


816